## Loading Training Data & Basic Information

In [ ]:
# If pandas is not installed, install it
%pip install pandas

In [8]:
import pandas as pd

# load the training data from the csv file
train = pd.read_csv("dataset-train-vf.csv")

# Shape of the training data
print("Training data shape: ", train.shape)

# Display feature names
print("Feature names: ", train.columns.tolist())

# Display the first few rows of the training data
train.head()

Training data shape:  (4480, 13)
Feature names:  ['ID', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'y']


,ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,y
0,1,NaN,62330,NaN,0.748,4.845455,30405,18.066667,2.807634,663180,NaN,C1,square
1,2,NaN,4370,NaN,0.858,1.072727,2445,1.266667,0.712986,49420,NaN,C2,square
2,3,0.000729,1449,196.3,0.841,0.172727,795,0.420000,0.112528,16240,NaN,C2,square
3,4,0.043499,24702,349.7,0.594,5.254545,9570,7.160000,2.417831,239680,0.430355,C3,circle
4,5,0.000972,1104,162.5,0.792,0.109091,570,0.320000,0.066930,12040,NaN,C3,square


### Important Notes

- The training data set has 13 columns: 1 ID column + 11 feature columns + 1 label column. The ID column will mostly be dropped since it is not a useful feature.
- For the features, all features are numerical (f1-f10) except for f11, which is categorical. We might need one hot encoding to deal with f11.
- Some columns show NaN values. We need to check how many NaNs per column to decide weather should we drop the feature or impute with some value (mean/median/mode).
- The label column has squares and circles. Therefore, it is a binary classification problem as stated before in the guidelines.

## Detailed Information

We will need to explore more information about the training dataset. Let's start with class distributions of the circle and square labels. Also, we will see the missing values count per column.

In [9]:
# Class distribution
print("Class distribution (absolute):")
print(train["y"].value_counts())

print("\n")

print("Class distribution (relative):")
print(train["y"].value_counts(normalize=True))

print("\n")

# Missing values count
missing_values = train.isnull().sum()
print("Missing values count:")
print(missing_values)


Class distribution (absolute):
y
square    4181
circle     299
Name: count, dtype: int64


Class distribution (relative):
y
square    0.933259
circle    0.066741
Name: proportion, dtype: float64


Missing values count:
ID        0
f1     1838
f2        0
f3     1384
f4        0
f5        0
f6        0
f7        0
f8        0
f9        0
f10    3912
f11       0
y         0
dtype: int64


### Important Notes

- Class imbalance: There are 4181 data points labeled "square" (~93%) and 299 data points labeled "circle" (~7). This means that: 
    - we shouldn't use accuracy, since itt will be misleading.
    - regardless, we will be using F1 score as required in the guidelines.
    - since "circle" is the minority class, we will consider it the positive class while the "square" is the negative class.
    - in the future, we will need to do some sampling strategy.

- Missing values: columns f1, f3, and f10 have many missing values. However, f1 and f3 columns have less than half of the data points missing while f10 is missing for almost all data points. Therefore, we might do the following
    - Impute the values in f1 and f3.
    - Drop column f10.

However, we it is better to analyze the data more. So, let's have a summary of the data columns:

In [10]:
# Get Info about data types
train.info()

# Summary stats for numeric features
display(train.describe())

# Distribution of categorical feature (only f11)
print(train["f11"].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4480 entries, 0 to 4479
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      4480 non-null   int64  
 1   f1      2642 non-null   float64
 2   f2      4480 non-null   int64  
 3   f3      3096 non-null   float64
 4   f4      4480 non-null   float64
 5   f5      4480 non-null   float64
 6   f6      4480 non-null   int64  
 7   f7      4480 non-null   float64
 8   f8      4480 non-null   float64
 9   f9      4480 non-null   int64  
 10  f10     568 non-null    float64
 11  f11     4480 non-null   object 
 12  y       4480 non-null   object 
dtypes: float64(7), int64(4), object(2)
memory usage: 455.1+ KB


,ID,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10
count,4480.000000,2642.000000,4.480000e+03,3096.000000,4480.000000,4480.000000,4480.000000,4480.000000,4480.000000,4.480000e+03,568.000000
mean,2240.500000,0.007127,2.559716e+04,234.375598,0.786005,3.162125,10897.995536,7.419467,1.804952,2.575234e+05,0.506391
std,1293.408932,0.009138,9.400564e+04,106.207714,0.163220,10.189312,25574.563533,27.248011,5.978266,7.739036e+05,0.289980
min,1.000000,0.000081,1.610000e+02,35.750000,0.062000,0.009091,105.000000,0.046667,0.006204,1.960000e+03,0.000224
25%,1120.750000,0.001397,2.783000e+03,170.300000,0.681000,0.400000,1530.000000,0.806667,0.252502,3.136000e+04,0.251716
50%,2240.500000,0.003321,7.590000e+03,218.075000,0.802500,1.004545,3885.000000,2.200000,0.600454,8.400000e+04,0.490897
75%,3360.250000,0.008586,2.270100e+04,271.050000,0.920000,2.581818,11070.000000,6.580000,1.471806,2.419200e+05,0.759281
max,4480.000000,0.044714,3.311839e+06,650.000000,1.000000,255.390909,642315.000000,959.953333,160.688556,2.339190e+07,0.999856


f11
C3    1152
C2    1133
C1    1110
C4    1085
Name: count, dtype: int64


### Important Notes

- ID feature: It is just an identifier.
- F1 and F3 feature: Both have some missing values but not exceeding half the samples.
- F10 feature: It has only 568 non-null value out of 4480 (14%). 
- F11 feature: The 4 categories in f11 are nicely balanced.
- Other features: All other features are fine with no major issue.


Therefore, we will do the following updates to the dataset:

1. Drop ID column.
2. Drop F10 feature because it has 86% missing values (NaN).

Note that imputing happens after splitting the date to prevent data leakage.



In [ ]:
%pip install scikit-learn

In [27]:
from sklearn.model_selection import train_test_split

# 1. & 2. Drop ID and F10
columns_to_drop = ["ID", "f10"]
data = train.drop(columns=columns_to_drop)

# Separate features (X) and target (y)
X = data.drop(columns=["y"])
y = data["y"]

# Split the data into training and validation sets, use stratified split due to calss imbalance
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training Set Dimensions: ", X_train.shape)
print("Validation Set Dimensions: ", X_val.shape)


Training Set Dimensions:  (3584, 10)
Validation Set Dimensions:  (896, 10)


Next steps are:

3. Features f1 - f9 will be used as numerical features (f1 and f3 will be imputed).
4. Apply one hot encoding on the categorical feature f11.
5. We will also create a logistic regression model as a baseline for future steps.

Note that this is just a baseline, we don't seek high performance now.

6. We will evaluate the logistic regression model

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Define numerical and categorical features
numeric_features = ["f1","f2","f3","f4","f5","f6","f7","f8","f9"]
categorical_features = ["f11"]

# 3. Preprocess f1 and f3 using a pipeline by imputing missing values with the median and scaling (standard scaling)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# 4. Apply one hot encoding on the categorical feature f11
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")), # Impute missing values with the most frequent value
    ("onehot", OneHotEncoder(handle_unknown="ignore")) # Apply one hot encoding
])

# Combine the transformers
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

# Create a logistic regression model
log_reg_model = LogisticRegression(
    class_weight="balanced",
    max_iter=1000,
    n_jobs=-1
)

# Create a pipeline
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", log_reg_model)
])

clf


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [32]:
from sklearn.metrics import classification_report

# 5. Fit the model
clf.fit(X_train, y_train)

# Make predictions on the validation set
y_val_pred = clf.predict(X_val)

# 6. Evaluate the model
print(classification_report(y_val, y_val_pred, digits = 4))

              precision    recall  f1-score   support

      circle     0.2426    0.6833    0.3581        60
      square     0.9739    0.8469    0.9060       836

    accuracy                         0.8359       896
   macro avg     0.6082    0.7651    0.6320       896
weighted avg     0.9249    0.8359    0.8693       896



d:\Projects\ICS485_Project\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


### Important Notes

- Based on the evaluation, we see that f1-score for the circle (minority) is 36% while the precision and recall are 24% and 68%, respectively.
- While for the squares, the precision is 97% and the recall is 85%.

This tells us that the logistic regression model is catching many circle points (68% recall) but it assigns many points as circles while they should be squares, meaning that it has many false positives (24% precision). This is mainly because the dataset is imbalanced. In the code, we assigned class weights for the logistic regression to "balanced", which made the model care more about the minority class, leading to low precision and high recall.

Let's check the confusion matrix before going on to future steps.

In [33]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_val, y_val_pred, labels=["circle", "square"])
cm


array([[ 41,  19],
       [128, 708]])